In [2]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

from pandas_datareader.data import DataReader
import yfinance as yf

from datetime import datetime

from urllib.request import urlopen, Request 
from bs4 import BeautifulSoup
import os
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [23]:
#US market data download
GSPC = DataReader('^GSPC', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
GSPC['SMA10'] = GSPC['Close'].rolling(10).mean()
GSPC['type']='index'
GSPC['ticker']="GSPC"
AAPL = DataReader('AAPL', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
AAPL['SMA10'] = AAPL['Close'].rolling(10).mean()
AAPL['type']="stock"
AAPL['ticker']="AAPL"
AMZN = DataReader('AMZN', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
AMZN['SMA10'] = AMZN['Close'].rolling(10).mean()
AMZN['type']="stock"
AMZN['ticker']="AMZN"
BABA = DataReader('BABA', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
BABA['SMA10'] = BABA['Close'].rolling(10).mean()
BABA['type']="stock"
BABA['ticker']="BABA"
GOOG = DataReader('GOOG', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
GOOG['SMA10'] = GOOG['Close'].rolling(10).mean()
GOOG['type']="stock"
GOOG['ticker']="GOOG"
TSLA = DataReader('TSLA', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
TSLA['SMA10'] = TSLA['Close'].rolling(10).mean()
TSLA['type']="stock"
TSLA['ticker']="TSLA"

US=pd.concat([GSPC, AAPL, AMZN, BABA, GOOG, TSLA])
US['region']='US'
US['latitude']=40.730610
US['longitude']=-73.935242

#HK market data download
HSI = DataReader('^HSI', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
HSI['SMA10'] = HSI['Close'].rolling(10).mean()
HSI['type']="index"
HSI['ticker']="HSI"
HK0700 = DataReader('0700.HK', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
HK0700['SMA10'] = HK0700['Close'].rolling(10).mean()
HK0700['type']="stock"
HK0700['ticker']="HK0700"
HK3690 = DataReader('3690.HK', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
HK3690['SMA10'] = HK3690['Close'].rolling(10).mean()
HK3690['type']="stock"
HK3690['ticker']="HK3690"
HK9988 = DataReader('9988.HK', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
HK9988['SMA10'] = HK9988['Close'].rolling(10).mean()
HK9988['type']="stock"
HK9988['ticker']="HK9988"

HK=pd.concat([HSI, HK0700, HK3690, HK9988])
HK['region']="HK"
HK['latitude']=22.302711 
HK['longitude']=114.177216

#Other market data download
JP225 = DataReader('^N225', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
JP225['SMA10'] = JP225['Close'].rolling(10).mean()
JP225['type']="index"
JP225['ticker']="JP225" 
JP225['region']="JP"
JP225['latitude']=35.652832
JP225['longitude']=139.839478

SS=DataReader('000001.SS', data_source = 'yahoo', start='2020-01-01', end=datetime.now())
SS['SMA10'] = SS['Close'].rolling(10).mean()
SS['type']="index"
SS['ticker']="SS" 
SS['region']="SS" 
SS['latitude']=31.224361 
SS['longitude']=121.469170 

raw=pd.concat([US, HK, JP225, SS])


In [31]:
raw.to_csv('C:/Stock_sourcedata/EDA/price.csv')

In [27]:
#scrap news from finwiz
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables={}
tickers =['AAPL', 'AMZN', 'BABA', 'GOOG', 'TSLA']
for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table

In [28]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element
        if len(date_scrape) == 1:
            time = date_scrape[0]

        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        ticker = file_name.split('_')[0]

        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time,text])

# print first 3 rows of news
parsed_news[:3] 


[['AAPL',
  'Jun-26-22',
  '11:00AM',
  'A cold dark place  Michael Burry thinks the market has plenty of room to plunge. But he finally sees value in these 4 stocks'],
 ['AAPL',
  'Jun-26-22',
  '09:28AM',
  '2 No-Brainer Warren Buffett Stocks to Buy Now and Hold Forever'],
 ['AAPL',
  'Jun-26-22',
  '09:01AM',
  'Market Plunge 2022: 3 Absolute Bargains Begging to Be Bought']]

In [29]:
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Set column names
columns = ['ticker', 'date', 'time', 'headline']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head()


,ticker,date,time,headline,neg,neu,pos,compound
0,AAPL,2022-06-26,11:00AM,A cold dark place Michael Burry thinks the ma...,0.0,0.871,0.129,0.4767
1,AAPL,2022-06-26,09:28AM,2 No-Brainer Warren Buffett Stocks to Buy Now ...,0.0,1.000,0.000,0.0000
2,AAPL,2022-06-26,09:01AM,Market Plunge 2022: 3 Absolute Bargains Beggin...,0.0,1.000,0.000,0.0000
3,AAPL,2022-06-26,08:21AM,Better Buy: Dividend Aristocrats vs. Beaten-Do...,0.0,0.707,0.293,0.4404
4,AAPL,2022-06-26,07:05AM,"Have $3,000? These 2 Stocks Could Be Bargain B...",0.0,0.859,0.141,0.2023


In [30]:
#transform sentiment_score
plt.rcParams['figure.figsize']=[10,6]
mean_scores=parsed_and_scored_news.groupby(['ticker','date']).mean()
mean_scores=mean_scores.unstack()
mean_scores=mean_scores.xs('compound',axis="columns").transpose()
sentiment_score=mean_scores.stack()
sentiment_score.head()

date        ticker
2022-06-06  BABA      0.072933
2022-06-07  BABA      0.197350
2022-06-08  BABA      0.173909
2022-06-09  BABA      0.094744
2022-06-10  BABA      0.122220
dtype: float64

In [35]:
parsed_and_scored_news.to_csv('C:/Stock_sourcedata/EDA/news.csv')
sentiment_score.to_csv('C:/Stock_sourcedata/EDA/sentiment.csv')